In [1]:
# Dependencies
import json
import numpy as np
import pandas as pd
import requests
from pprint import pprint 

In [2]:
#Get a list of countries with data on the api:
Country_list_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldwide-pollution&facet=country&facet=filename&refine.country=Argentina&refine.country=Albania&refine.country=Algeria&refine.country=Angola&refine.country=Armenia&refine.country=Australia&refine.country=Austria&refine.country=Azerbaijan&refine.country=Bangladesh&refine.country=Belarus&refine.country=Belgium&refine.country=Belize&refine.country=Benin&refine.country=Bhutan&refine.country=Bolivia&refine.country=Bosnia+and+Herzegovina&refine.country=Botswana&refine.country=Brazil&refine.country=Brunei+Darussalam&refine.country=Bulgaria&refine.country=Burkina+Faso&refine.country=Namibia&refine.country=Myanmar&refine.country=Mozambique&refine.country=Morocco&refine.country=Montenegro&refine.country=Moldova&refine.country=Mongolia&refine.country=Mexico&refine.country=Mauritania&refine.country=Mali&refine.country=Malaysia&refine.country=Malawi&refine.country=Madagascar&refine.country=Afghanistan&refine.country=Cambodia&refine.country=Cameroon&refine.country=Burundi&refine.country=Canada&refine.country=Chile&refine.country=Chad&refine.country=Central+African+Republic&refine.country=China&refine.country=Colombia&refine.country=Costa+Rica&refine.country=C%C3%B4te+d%27Ivoire&refine.country=Croatia&refine.country=Cuba&refine.country=Cyprus&refine.country=Czech+Republic&refine.country=Democratic+Republic+of+the+Congo&refine.country=Denmark&refine.country=Dominican+Republic&refine.country=Ecuador&refine.country=Egypt&refine.country=Macedonia&refine.country=Luxembourg&refine.country=Lithuania&refine.country=Libya&refine.country=Liberia&refine.country=Lesotho&refine.country=Lebanon&refine.country=Latvia&refine.country=Lao+PDR&refine.country=Kyrgyzstan&refine.country=Kuwait&refine.country=Kosovo&refine.country=Kenya&refine.country=Kazakhstan&refine.country=Jordan&refine.country=Japan&refine.country=Jamaica&refine.country=Italy&refine.country=Ireland&refine.country=Iraq&refine.country=Israel&refine.country=Iran&refine.country=Indonesia&refine.country=India&refine.country=Iceland&refine.country=Hungary&refine.country=Honduras&refine.country=Haiti&refine.country=Guyana&refine.country=Guinea-Bissau&refine.country=Guinea&refine.country=Guatemala&refine.country=Greenland&refine.country=Greece&refine.country=Germany&refine.country=Ghana&refine.country=Georgia&refine.country=Gabon&refine.country=France&refine.country=Ethiopia&refine.country=Estonia&refine.country=Finland&refine.country=Eritrea&refine.country=Equatorial+Guinea&refine.country=El+Salvador'
country_response = requests.get(Country_list_url).json()
countries_list = country_response['parameters']['refine']['country']

In [ ]:
#loop:
rows = []
for country in countries_list:
    hits_url = 'https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldwide-pollution&facet=country&facet=filename&refine.country={}'.format(country)
    hits_response = requests.get(hits_url).json()
    num_hits = hits_response['nhits']
    url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=worldwide-pollution&rows={}&facet=country&facet=filename&refine.country={}".format(num_hits, country)
    response = requests.get(url).json()
#Run a for loop on the list produced by response['records']
#Note each element of the list is a dictionary of records for a spectic city (i.e filename)
    try:
        for x in range(len(response['records'])):
            try:
                city = response['records'][x]['fields']['filename']
            except KeyError:
                city = ''
            country = response['records'][x]['fields']['country']
            data_str = response['records'][x]['fields']['data']
            country_city_data = json.loads(data_str)
            try:
# Reference the pollutants key for the country_data dictionary
#The pollutant key produces a dictionary of pollutant 'keys' 
# with 'values' as dictionaries with relevant information for each pollutant 'key'
#Within the dictionary of a specific pollutant key access the 'values' key which is pair with the pollutant value for that key.
                poll_CO_val = country_city_data['pollutants']['CO']['aqi']['epa']['values']
                poll_O3_val = country_city_data['pollutants']['O3']['aqi']['epa']['values']
                poll_NO2_val = country_city_data['pollutants']['NO2']['aqi']['epa']['values']
                poll_PM25_val = country_city_data['pollutants']['PM25']['aqi']['epa']['values']
                longitude = country_city_data['location']['longitude']
                latitude = country_city_data['location']['latitude']
            except:
                pass
            rows.append({'Country': country, 'City': city, 'Value CO': poll_CO_val[0], 'Value O3': poll_O3_val[0],  'Value NO2': poll_NO2_val[0],'Value PM25': poll_PM25_val[0], 'Longitude': longitude, 'Latitude': latitude})
    except KeyError:
        pass

In [ ]:
World_Pollution_df = pd.DataFrame(rows)
World_Pollution_df.head()

In [ ]:
World_Pollution_df.to_csv("../DataFrames/World_Pollution_df.csv", header=True)